In [66]:
from langchain_groq import ChatGroq
import uuid
import chromadb
import pandas as pd
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.llms import OpenAI


In [67]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [68]:
print(response.content)


The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [69]:
loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/110299876444512966-regional-process-manager-source-to-pay-operations-english?location=Poland")
page_data = loader.load().pop().page_content
print(page_data)  # Display the scraped job data


Regional Process Manager, Source to Pay Operations (English) — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results103  jobs matchedSenior Software Engineer, AI Studio, Google CloudWarsaw, PolandRegional Process Manager, Source to Pay Operations (English)Kraków, PolandSenior Software Engineer, Kubernetes, Google CloudWarsaw, PolandCloud Engineer, SAP AnalyticsWarsaw, PolandSoftware Engineer, Early Career, Cloud AIWarsaw, PolandSoftware Engineer III, Chrome MetricsWarsaw, PolandSoftware Engineer, Infrastructure, CommerceWarsaw, PolandSoftware Engineer, Google Cloud PlatformWarsaw, PolandPrincipal Software Engineer, Google Compute Engine Control PlaneWarsaw, PolandSoftware Engineer, Early Career, CampusWarsaw, Po

In [70]:
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job posting and return them in JSON format containing the 
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):    
    """
)

# Initialize LLM
# llm = OpenAI(model_name="text-davinci-003")  # Adjust model as necessary

# Combine prompt and LLM into a chain
chain_extract = prompt_extract | llm

# Extract job details
res = chain_extract.invoke(input={'page_data': page_data})

# Parse JSON Response
json_parser = JsonOutputParser()
jobs = json_parser.parse(res.content)

# Display Extracted Job Postings
jobs


{'role': 'Regional Process Manager, Source to Pay Operations (English)',
 'experience': 'Mid',
 'skills': ["Bachelor's degree in Business, Finance, Engineering, or Operations, or equivalent practical experience",
  '4 years of experience in management consulting, or an operations-related function; or 2 years of experience with an advanced degree in business, accounting, or finance',
  'Ability to communicate in English fluently',
  'Experience with accounting/finance ERP systems and business processes',
  'Experience with an extended workforce service provider in operations, source-to-pay, or other relevant areas',
  'Experience leading complex, global operational initiatives and system implementations'],
 'description': 'Manage the company’s global supplier enrollment, invoicing, and payment operations executed by our partner in onshore and offshore centers to deliver process excellence focusing on efficiency, quality, controls, and compliance to local regulations. Analyze data-sets t

In [71]:
type(jobs)

dict

In [72]:
# Ensure you have a CSV file named `my_portfolio.csv` with "Techstack" and "Links" columns
# Sample portfolio data - replace with your actual CSV file if necessary
df = pd.read_csv("my_portfolio.csv")
df




,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [73]:
# Initialize ChromaDB PersistentClient
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

# Add data to the collection if it's empty
if collection.count() == 0:
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )



In [74]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [75]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [76]:
# job = {
#     "role": "Regional Process Manager, Source to Pay Operations (English)",
#     "skills": [
#         "Management consulting",
#         "Operations-related function",
#         "Accounting/finance ERP systems",
#         "Business processes",
#         "System controls",
#         "Leadership",
#         "Communication",
#         "Problem-solving",
#         "Attention to detail",
#         "Organization"
#     ],
#     "description": "Manage the company's global supplier enrollment, invoicing, and payment operations. Analyze data-sets to identify, prioritize, and address risks and opportunities. Partner with various business and Finance teams to identify and gather business requirements, design solutions, and implement process improvements."
# }


In [77]:
links = collection.query(query_texts=jobs['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}]]

In [78]:
jobs

{'role': 'Regional Process Manager, Source to Pay Operations (English)',
 'experience': 'Mid',
 'skills': ["Bachelor's degree in Business, Finance, Engineering, or Operations, or equivalent practical experience",
  '4 years of experience in management consulting, or an operations-related function; or 2 years of experience with an advanced degree in business, accounting, or finance',
  'Ability to communicate in English fluently',
  'Experience with accounting/finance ERP systems and business processes',
  'Experience with an extended workforce service provider in operations, source-to-pay, or other relevant areas',
  'Experience leading complex, global operational initiatives and system implementations'],
 'description': 'Manage the company’s global supplier enrollment, invoicing, and payment operations executed by our partner in onshore and offshore centers to deliver process excellence focusing on efficiency, quality, controls, and compliance to local regulations. Analyze data-sets t

In [79]:

jobs['skills']

["Bachelor's degree in Business, Finance, Engineering, or Operations, or equivalent practical experience",
 '4 years of experience in management consulting, or an operations-related function; or 2 years of experience with an advanced degree in business, accounting, or finance',
 'Ability to communicate in English fluently',
 'Experience with accounting/finance ERP systems and business processes',
 'Experience with an extended workforce service provider in operations, source-to-pay, or other relevant areas',
 'Experience leading complex, global operational initiatives and system implementations']

In [86]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Abdul Ghaffar Ansari, a Master's student in Artificial Intelligence at Brandenburgische Technische Universität, Cottbus, Germany, 
    with a strong background in AI, MLOps, and data analytics. You are currently seeking a working student position to leverage your 
    technical expertise and hands-on experience in cloud infrastructure, machine learning, and scalable applications. 
    Your Contact Information: LinkedIn: https://www.linkedin.com/in/abdul-ghaffar-ansari-ai/ , and your GitHub: https://www.github.com/abdulghaffaransari.
    
    
    Use the information provided in your profile to write a professional email expressing your interest in the position mentioned above. 
    Mention relevant skills and experiences that align with the role and describe how you can contribute to the organization.
    Also, highlight your certifications and projects,that are highly relevant to job posting.
    Ensure the email is concise, professional, and tailored to the job description. Include the most relevant links from the portfolio: {link_list}.
    Do not provide a preamble.
    
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({
    "job_description": str(jobs),
    "link_list": links
})
print(res.content)


Subject: Application for Regional Process Manager, Source to Pay Operations

Dear Hiring Manager,

I am excited to express my interest in the Regional Process Manager, Source to Pay Operations position at your esteemed organization. As a Master's student in Artificial Intelligence at Brandenburgische Technische Universität, Cottbus, Germany, I am confident that my technical expertise and hands-on experience in data analytics, cloud infrastructure, and scalable applications can be leveraged to drive process excellence in your global supplier enrollment, invoicing, and payment operations.

With a strong background in AI and MLOps, I possess excellent analytical skills, which can be utilized to analyze data-sets, identify risks and opportunities, and drive standardization and continuous improvement. My experience with machine learning and data analytics can help optimize business processes, ensuring efficiency, quality, controls, and compliance to local regulations. Although my primary ex